## Make the aliens move
Now that we have made the fleet of aliens, next task is to make them move on the screen. To make it a bit more interesting we will make the fleet move right across the screen until it hits the edge, and then make it drop a set amount and move in the other direction. We’ll continue this movement until all aliens have been shot down or one reaches the bottom of the screen.


#### Making them move right
To move the aliens, we’ll use an `update()` method in *alien.py*, which we’ll call for each alien in the group of aliens. So add the following function to the class.

In [ ]:
def update(self):
  """Move the alien right."""
  self.rect.x += 1


In the main **while** loop in *alien_invasion.py*, we have calls to update the ship and bullets. Now we need to update the position of each alien as well:


In [ ]:
  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update()
    gf.update_bullets(bullets)
    gf.update_aliens(aliens)
    gf.update_screen(bg_img, screen, ship, aliens, bullets)


We update the aliens’ positions after the bullets have been updated,
because we’ll soon be checking to see whether any bullets hit any aliens.

Finally, add the new function `update_aliens()` at the end of the file *game_functions.py*.


In [ ]:
def update_aliens(aliens):
  """Update the postions of all aliens in the fleet."""
  aliens.update()


We use the update() method on the aliens group, which automatically
calls each alien’s update() method. When you run Alien Invasion now, you
should see the fleet move right and disappear off the side of the screen.


Next we need to have an attribute to keep track in which direction is the fleet supposed to move at that instant i.e. left or right. So we add the `fleet_direction` attribute in *settings.py*. 

In [ ]:
# fleet_direction of 1 represents right; -1 represents left.
self.fleet_direction = 1

To implement the setting `fleet_direction`, we could use a text value, such as 'left' or 'right', but we’d end up with if-elif statements testing for the fleet direction. Instead, because we have only two directions to deal with, let’s use the values 1 and −1 and switch between them each time the fleet changes direction.

We also need a method to check whether an alien is at either edge, and we need to modify `update()` to allow each alien to move in the appropriate direction. So we add this `check_edges()` function in the Alien class in *alien.py* and also modify update accordingly. 


In [ ]:
def check_edges(self):
  """Return True if alien is at edge of screen."""
  screen_rect = self.screen.get_rect() # get rect object of the screen
  if self.rect.right >= screen_rect.right:
    return True
  elif self.rect.left <= 0:
    return True

def update(self):
  """Move the alien right or left."""
  self.rect.x += self.ai_settings.fleet_direction


We can call the new method `check_edges()` on any alien to see if it’s at the left or right edge. The alien is at the right edge if the right attribute of its rect is greater than or equal to the right attribute of the screen’s rect. It’s at the left edge if its left value is less than or equal to 0.

We also modify `update()` to allow the fleet in both directions depending on the value of **fleet_direction**.

When an alien reaches the edge, the entire fleet needs to drop down and change direction. We therefore need to make some substantial changes in *game_functions.py* because that’s where we check to see if any aliens are at the left or right edge.

We’ll make this happen by writing the functions `check_fleet_edges()` and `change_fleet_direction()` , and then modifying `update_aliens()`

In [ ]:
def check_fleet_edges(ai_settings, aliens):
  """Respond appropriately if any aliens have reached an edge."""
  for alien in aliens.sprites():
    #iterate through all the aliens and check if any have reached the edge
    if alien.check_edges():
      change_fleet_direction(ai_settings, aliens)
      break

def change_fleet_direction(ai_settings, aliens):
  """Drop the entire fleet and change the fleet's direction."""
  for alien in aliens.sprites():
    alien.rect.y += 5 # make the aliens move down
  ai_settings.fleet_direction *= -1 # change the direction of movement

def update_aliens(ai_settings, aliens):
  """
  Check if the fleet is at an edge,
  and then update the postions of all aliens in the fleet.
  """
  check_fleet_edges(ai_settings, aliens)
  aliens.update()


In `check_fleet_edges()`, we loop through the fleet and call `check_edges()` on each alien. If `check_edges()` returns *True*, we know an alien is at an edge and the whole fleet needs to change direction, so we call `change_fleet_direction()` and break out of the loop. In `change_fleet_direction()`, we loop through all
the aliens and drop each one. Then we change the value of **fleet_direction** by multiplying its current value by −1.

This function needs an ai_settings parameter, so we include an argument for ai_settings in the call to `update_aliens()`:



In [ ]:
  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update()
    gf.update_bullets(bullets)
    gf.update_aliens(ai_settings, aliens)
    gf.update_screen(bg_img, screen, ship, aliens, bullets)


If you run the game now, the fleet should move back and forth between
the edges of the screen and drop down every time it hits an edge. Next step is to start shooting the aliens

## Shooting the Aliens
In order to make the bullets shoot the aliens down we need to check for *collisions* between aliens and bullets. *Collisions* happen when game elements overlap. We’ll use the method `sprite.groupcollide()` to look for collisions between members of two groups (aliens and bullets). 

The method will compare each bullet’s rect with each alien’s rect and will return a dictionary containing the bullets and aliens that have collided. Each key in the dictionary will be a bullet, and the corresponding value will be of the alien that was hit. With the sprite class there many methods availble to detect collisions between 2 sprites, 2 groups, sprite and group. If you want you can look up these in the [documentation](https://www.pygame.org/docs/ref/sprite.html#pygame.sprite.spritecollide). Even if we are not using sprites you could use `pygame.Rect.colliderect(Rect1, Rect2)` to detect if 2 rectangles overlap. 

We are going to add code for doing this in the `update_bullets()` function in *game_functions.py*

In [ ]:
def update_bullets(aliens, bullets):
  """Update position of bullets and get rid of old bullets."""
  --snip--
  # Check for any bullets that have hit aliens.
  # If so, get rid of the bullet and the alien.
  collisions = pygame.sprite.groupcollide(bullets, aliens, True, True)


The new line we added loops through each bullet in the group bullets
and then loops through each alien in the group aliens. Whenever the rects of a bullet and alien overlap, `groupcollide()` adds a key-value pair to the dictionary it returns. The two **True** arguments tell Pygame whether to delete the bullets and aliens that have collided. (If you wanted to make a high-powered bullet as a powerup that’s able to travel to the top of the screen, destroying every alien in its path, you could set the first Boolean argument to False and keep the second Boolean argument set to True. The aliens hit would disappear, but all bullets would stay active until they disappeared off the top of the screen.)

We now also need the aliens group in this function so we are going to have pass it when calling the function in *alien_invasion.py*.



In [ ]:
  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update()
    gf.update_bullets(aliens, bullets)
    gf.update_aliens(ai_settings, aliens)
    gf.update_screen(bg_img, screen, ship, aliens, bullets)


Now if you'll run the code you should be able to shoot the aliens as they move down. We have now almost completed the game only a few things to make it look complete are left.

To continue the game a new fleet should appear everytime all the aliens have been shot.
We modify the `update_bullets()` for this:

In [ ]:
def update_bullets(ai_settings, screen, ship, aliens, bullets):
  --snip--
  # Check for any bullets that have hit aliens.
  # If so, get rid of the bullet and the alien.
  collisions = pygame.sprite.groupcollide(bullets, aliens, True, True)
  
  if len(aliens) == 0:
    # Destroy existing bullets and create new fleet.
    bullets.empty()
    create_fleet(ai_settings, screen, ship, aliens)


We check whether the group **aliens** is empty. If it is, we get rid
of any existing bullets by using the `empty()` method, which removes all the remaining *sprites* from a group. We also call `create_fleet()`, which fills the screen with aliens again.

The definition of `update_bullets()` now has the additional parameters ai_settings, screen, and ship , so we need to update the call to `update_bullets()` in *alien_invasion.py*:



In [ ]:
  # Start the main loop for the game.
  while True:
    gf.check_events(ai_settings, screen, ship, bullets)
    ship.update()
    gf.update_bullets(ai_settings, screen, ship, aliens, bullets)
    gf.update_aliens(ai_settings, aliens)
    gf.update_screen(bg_img, screen, ship, aliens, bullets)


Let’s also **refactor** `update_bullets()` so it’s not doing so many different tasks. We’ll move the code for dealing with bullet-alien collisions to a separate function:


In [ ]:
def update_bullets(ai_settings, screen, ship, aliens, bullets):
  --snip--
  # Get rid of bullets that have disappeared.
  for bullet in bullets.copy():
    if bullet.rect.bottom <= 0:
      bullets.remove(bullet)

  check_bullet_alien_collisions(ai_settings, screen, ship, aliens, bullets)

def check_bullet_alien_collisions(ai_settings, screen, ship, aliens, bullets):
  """Respond to bullet-alien collisions."""
  # Remove any bullets and aliens that have collided.
  collisions = pygame.sprite.groupcollide(bullets, aliens, True, True)

  if len(aliens) == 0:
    # Destroy existing bullets and create new fleet.
    bullets.empty()
    create_fleet(ai_settings, screen, ship, aliens)


## Ending the Game
Now we will finally code for ending the game. The game ends when either the aliens hit the bottom or the ship. 

First we will program to detect collisions between the ship and the aliens. For this we will use the method `spritecollideany()`. It takes two arguements, a *sprite* and a *group* and then checks whether any member of the *group* has collided with the ship. 

To end the game we create a function `game_over()` and call it in `update_aliens()` as follows:

In [ ]:
--snip--
from time import sleep

--snip--

def game_over(ai_settings, screen):
  gameover_img = pygame.font.SysFont(None, 48).render("GAME OVER", True, (240,240,240))
  # get rect object of the font
  gameover_rect = gameover_img.get_rect()
  #postion the rect in center at top
  gameover_rect.centerx = ai_settings.screen_width/2
  gameover_rect.top = 20
  #draw the text on the screen
  screen.blit(gameover_img, gameover_rect)
  # display the last drawn screen
  pygame.display.flip()
  # wait for 3 seconds and then exit the game
  sleep(3)
  sys.exit(0)

def update_aliens(ai_settings, screen, aliens, ship):
  """
  Check if the fleet is at an edge,
  and then update the postions of all aliens in the fleet.
  """
  check_fleet_edges(ai_settings, aliens)
  aliens.update()
  # Look for alien-ship collisions.
  if pygame.sprite.spritecollideany(ship, aliens):
    game_over(ai_settings, screen)

--snip--

Using the `spritecollideany()` to check if any alien has collided with the ship. Then we call the `game_over` function. The aim of the function is to freeze the game for 3 seconds with "GAME OVER" written at the top and then exit.

Here we use `pygame.font` to display the text "GAME OVER" on the screen. The *None* argument tells Pygame to use the default font(you can here specify a particular font as well) , and 48 determines the size of the text. The `render()` method then turns the text into an image which can then be treated just like any other image.It also takes a Boolean value to turn antialiasing on or off (antialiasing makes the edges of the text smoother). The remaining arguments are the font color and background color.

We then got the rect object from this text image, specifide its position and drew it on the screen using the `blit()` method. Finally we use the `display.flip()` method to draw the screen with this text. 

To freeze the game for 3 seconds we use `sleep` method which we imported from the `time` module. After that we use `sys.exit(0)` to exit the game.




We also need to end the game if aliens reach the bottom of the screen. We just need to check if the bottom of the alien's rect does not excedd the screen's height. We add a function `check_aliens_bottom()` in *game_functions.py* and use it in `update_aliens()`:

In [ ]:
def check_aliens_bottom(screen,aliens):
  """Check if any aliens have reached the bottom of the screen."""
  screen_rect = screen.get_rect()
  for alien in aliens.sprites():
    if alien.rect.bottom >= screen_rect.bottom:
      return True

def update_aliens(ai_settings, screen, aliens, ship):
  --snip--
  # Look for alien-ship collisions or whether an alien has reached the bottom
  if pygame.sprite.spritecollideany(ship, aliens) or check_aliens_bottom(screen,aliens):
    game_over(ai_settings, screen)

The `check_aliens_bottom()` checks and returns **True** if any alien is at the bottom. 



With this we have got a somewhat complete game. There is a fleet of aliens heading down following a certain pattern. You can shoot those aliens down from the ship. The fleet continues to repopulate each time you shoot them down, continuing the game. The game ends if any of the aliens reach the bottom or hit the ship.